# Reading Data Files with pandas

There are three data files. Each file contains the reaction times (RTs) from 10 trials of a relatively simple task in which participants had to indicate which direction a briefly-presented arrow was pointing. The RTs are in seconds (s). Each file contains the RTs from a different participant. In each file there are three columns. You can determine what the columns are by looking at the first row (header) of each file.

In this exercise, we will read in the data from the three files in the `data` directory: `s1.csv`, `s2.csv`, and `s3.csv`, and combine them into a single DataFrame. We will save that DataFrame, and then then calculate the mean RT for each participant.

## Read in the Data

To start the process, I typed the first two lines of the prompt below:

```python
# read in three files from the data folder, whose names start with "s" and end in "csv"
# concatenate them into one dataframe
```

After I hit `Enter`, Copilot suggested the third line of the prompt, which I accepted by hitting `Tab`:

```python
# write the result to a csv file
```

The cells below all reflect prompts written on the basis of the instructions above, and code generated entirely by Copilot. While your experience may be different, in writing this lesson we only had to type the first 2-line prompt, and Copilot generated not only the code but the other prompts/comments. 

However, you have to get used to the flow of working with Copilot. Copilot encourages a good coding style, and so it will sometimes only generate code if you hit `Enter` twice, so that there's an empty line between your prompt and the code. The same is true *after* a line of code: Copilot may force you to hit `Enter` twice before it generates the next prompt or line of code.

After I typed the first two lines of the prompt below, and accepted the third line of the prompt (which was suggested by Copilot), I had to hit `Enter` twice to get the first line of code (`import pandas as pd`) generated.

Although we intended to only read and combine the data files, after I typed the first two lines of the prompt and hit `Enter`, Copilot added a third suggested line to the prompt after I hit `Enter` the first time, which was to `write the dataframe to a csv file`. This is pretty cool, since that was also part of the instructions above! Is Copilot a mind-reader? No, it's just a very good predictor of what you might want to do next, based on the prompt you've written. It's a pretty common task in data science to read in a set of individual data files, concatenate them into a single DataFrame, and then save that DataFrame to a file for later use. So, Copilot is just doing what it's trained to do, which is to predict what you might want to do next, based on the prompt you've written.

In [1]:
import pandas as pd
import glob

# read in three files from the data folder, whose names start with "s" and end in "csv"
# concatenate them into one dataframe

data_files = glob.glob('data/s*.csv')
df = pd.concat((pd.read_csv(f) for f in data_files), ignore_index=True)

# write the result to a csv file
df.to_csv('data/combined_data.csv', index=False)


The code ran without errors, but we didn't ask for any output. Next we'll to some checks to confirm that the code ran correctly, as described in the instructions. We'll start by checking the number of columns in the DataFrame. As noted above, there should be three columns.

In [2]:

# show the first5 5 rows of the data frame
df.head(5)

,participantID,trial,RT
0,s2,1,0.433094
1,s2,2,0.392526
2,s2,3,0.396831
3,s2,4,0.417988
4,s2,5,0.371810


We see that the DataFrame has 3 columns, as expected. Next, we'll check that we get the expected number of rows (30) and columns (3):

In [3]:
# check that we have the epxected numebrs of rows(30) and columns(3)
df.shape

(30, 3)

We also were instructed to raise an error message if the number of rows or columns is incorrect:

In [4]:
# raise an error message if number of rows is not 30 or number of columns is not 3
# make the if statement more pythonic by combining the two conditions into one
#if df.shape[0] != 30 or df.shape[1] != 3:
if df.shape != (30, 3):
    raise ValueError('Dataframe does not have the expected shape of (30, 3)')

### Test Your Test
This generates no output. This is a Good Thing, because it means that the number of rows and columns is correct. However, it's a bit dangerous to assume no news is good news, because no news could also mean your code is not working. So, we should test that the error message is generated if we change the expected number of rows or columns.

We don't want to actually remove any rows from the DataFrame, but we can use **slicing** to create a view of the DataFrame that has fewer rows:

In [12]:
# create a slice of df with only 10 rows
df_slice = df.iloc[:10, :]
print(df_slice.shape)
# now same if statement on df_slice
# assert df_slice.shape == (10, 3)
if df_slice.shape != (30, 3):
    raise ValueError('Dataframe slice does not have the expected shape of (30, 3)')


SyntaxError: invalid syntax (1809890142.py, line 5)

Now we get an `AssertionError`, which is a Good Thing because it confirms that our error-checking code is working. We can do the same thing to test the error-checking code for the number of columns:

In [14]:
df_slice = df.iloc[:, :2]
# same if statement on df_slice
if df_slice.shape != (30, 3):
    raise ValueError('Dataframe slice does not have the expected shape of (30, 3)')

ValueError: Dataframe slice does not have the expected shape of (30, 3)

So yes, our code will throw errors if the number of rows or columns is incorrect. 

## Calculating the Mean RT for Each Participant... and Our First Bug
Our next instruction is to calculate the mean RT for each participant. Let's prompt Copilot to do that:

In [22]:
# calculate mean rt for each participant
mean_rt = df.groupby('participantID')['RT'].mean()
# print(df.groupby('participantID').mean())
print(mean_rt)

participantID
s1    0.389548
s2    0.444785
s3    0.446009
Name: RT, dtype: float64


In [23]:
df.sample(5)

,participantID,trial,RT
6,s2,7,0.411051
5,s2,6,0.659228
24,s1,5,0.437765
3,s2,4,0.417988
26,s1,7,0.400544


---
## Debugging Copilot-Generated Code

Typically, when you get a long, scary error message like the one above, you can ignore a lof of what is in the middle. The most important parts are the last line, which tells you what the error is, and the first lines, which usually indicate what line in the code you tried to run caused the error. What's in between is the *stack trace*, which is a list of all the functions that were called in the process of trying to run the code. But most of the time, the error is a result of the code you wrote (the first lines), not the code in the underlying Python functions that your code called (the middle lines).

In this case, we see a `KeyError: 'participant'` at the bottom of the error message. Recall that Python dictionaries are sets of **key**-value pairs. The keys are the names of the columns, and the values are the data in those columns. 

You can think of a pandas DataFrame as a dictionary in which the column names are keys, and the values in that column are its values. This is a common way that pandas functions refer to column names and their values. So the error message above indicates that the code tried to access a column called `subject` in the DataFrame, but that column doesn't exist. We can see at the top of the error message that the code that caused the error is: `---> 2 df.groupby('participant').mean()`. This means it's line 2 of the cell above that caused the error. 

As a novice coder, however, you might not know or remember that pandas DataFrames represent column names and data as dictionary key-value pairs. If you don't understand what an error means, you can use Copilot to help you!

### Using Copilot to Explain Errors and Suggest Fixes

Somewhat counterintuitively, even though Copilot can generate code with bugs, it can also provide feedback on errors in the code it generated. 

Copilot has a few features that can help you diagnose and fix your code. The one we have found most effective is Copilot Chat. For this reason, we won't describe other methods, such as Copilot's "Fix" feature, which is described in the [Copilot documentation](https://copilot.github.com/#/guide).

### ![](images/chat_icon.png)  Copilot Chat

 Another, really useful option is the `Chat` option, which you can option by clicking the `Chat` icon (shown above) in the Activity Bar (left side of VS Code). This allows you to have a conversation with Copilot about what you want to do. If you've used [Chat-GPT](https://chat.openai.com/) before, this will be familiar to you as it's essentially the same type of natural-language interface.

There are different approaches to using Copilot Chat to help fix an error. One is to click in the cell that generates the error, and then click in the Copilot Chat input box, and typing: 

```
the selected cell generates an error. how can i fix this error?
``` 

In response to this prompt for the present error, I got the following response:

>If you are getting an error with the selected cell, there could be a few reasons why. Here are some possible solutions:
>
>1. Check that the 'participant' column is present in the dataframe. You can do this by printing out the dataframe using the `print()` function and checking that the 'participant' column is present.
>
>2. Check that the 'participant' column is spelled correctly. If the column is misspelled, you will get a "KeyError" when trying to group by that column.
>
>3. Check that the 'participant' column is of the correct data type. If the 'participant' column is not a string or categorical data type, you may need to convert it using the `astype()` method.
>
>4. Check that the dataframe contains numerical columns. If the dataframe only contains non-numerical columns, you will get a "DataError" when trying to calculate the mean.
>
>5. Check that the dataframe does not contain missing values. If the dataframe contains missing values, you may need to remove them using the `dropna()` method before calculating the mean.

### Finding the Error with Copilot's Help

These are actually really good responses, and they are provided in a logical order to go through in debugging. We could work through them in order, but — spoiler alert — the first suggestion actually leads to a solution. 

### Check that the 'participant' Column is Present in the DataFrame
The first thing to do is to check that the column exists, and is spelled correctly. So let's start with that. Do you remember how to check the names of the columns in a pandas DataFrame? If not, you can write a Copilot prompt to help you out. In the cell below, write a prompt that will print the names of the columns in the DataFrame. Then run the cell, and see what happens.

In [24]:
# check the names of the columns in df
df.columns

Index(['participantID', 'trial', 'RT'], dtype='object')

Another option is to look at the first few rows of the DataFrame, which includes the column names:

In [25]:
# check the first 3 rows of the dataframe
df.head(3)

,participantID,trial,RT
0,s2,1,0.433094
1,s2,2,0.392526
2,s2,3,0.396831


A third option, when using Jupyter notebooks with VS Code, is to click on the `Variables` button in the toolbar at the top of the notebook window. This will pop up a variable viewer in sub-window below your notebook. You can click on the variable names to see their values. For DataFrames, it actually shows a list of the columns in the window, and you can double-click on the variable name to see the contents of the DataFrame in another window, the **Data Viewer**. This view is similar to a spreadsheet. In fact, you can directly edit values in the Data Viewer. *You should never directly edit values like this*, however. Any steps you do manually are not documented in your code, and are not reproducible.  

The screenshot below shows the variables and Data Viewer for the current context.

![](images/inspectors.png)

### The Solution

Using any of the three approaches above, when we look at the column names, we see that they are `participantID`, `trial`, and `RT`. The code that generated the error was trying to access a column called `participant`, which doesn't exist. It should be `participantID`. So we need to change the code to access the correct column name. Or, engineer our prompt to do so.

<div class="alert alert-block alert-info">
This is an important lesson on prompt engineering. You're more likely to get working code if you use the actual variable names in the prompt, rather than expecting Copilot to make inferences about what variable you're referring to based on the context.
</div>

In [26]:
# --- IGNORE ---
# find mean for each column grouped by participantID
df.groupby('participantID').mean()

,trial,RT
participantID,,
s1,5.5,0.389548
s2,5.5,0.444785
s3,5.5,0.446009


This looks good, however the code is providing means for both columns in the DataFrame (`trial` and `RT`), not just for `RT` (sometimes the same prompt actually does select only `RT` but we'll explore when it doesn't). We can add to our prompt to tell it not to include `trial` in the output:

In [29]:
# don;t include trial number in the mean calculation
df.groupby('participantID')[['RT']].mean()
# but make the code so that it is directly dropping trial number column
df.groupby('participantID').mean().drop(columns=['trial'])

,RT
participantID,
s1,0.389548
s2,0.444785
s3,0.446009


The above generated code does what we want. However, from the perspectives of coding style and efficiency, it's not optimal. Python executes this **chained command** from left to right. So, it first computes the mean for each column in the DataFrame, and then drops the column `trial`.

It seems unnecessary to compute the mean for `trial` and then drop it. This isn't really Copilot's fault — we did explicitly tell it not to show the mean for trial, but it's not smart enough to know that we don't want to compute it in the first place; it seems to have interpreted our prompt as a literal sequence of commands.

We can modify the prompt in a way that generates more efficient code, by being specific about the column that we do want, rather than what we don't want:

In [31]:
# don't include trial number in the calculation
df.groupby('participantID').mean().loc[:, df.columns != 'trial']
# no ignore the trial number completely
df.loc[:, df.columns != 'trial'].groupby('participantID').mean()
# no looking for something like df.groupby('participantID')[['RT']].mean()
# code:
df.groupby('participantID')['RT'].mean()


IndexError: Boolean index has wrong length: 3 instead of 2

By way of showing how sensitive Copilot is to the structure of your prompt, a slightly different (and arguably more logical) phrasing of the prompt above generates the less-efficient code:

In [32]:
df.groupby('participantID')['RT'].mean()
# do not compute mean for trial RT and then drop it though, give the new code
df.groupby('participantID')['RT'].mean()

participantID
s1    0.389548
s2    0.444785
s3    0.446009
Name: RT, dtype: float64

<div class="alert alert-info">
One thing you may notice is that the result of the last command above is nicely-formatted when it is displayed, whereas the one before it is in a more "raw" format. This is not really important here, but it's worth understanding why the difference occurs. When you call a pandas DataFrame it prints in a nicely formatted output. However, when you call a pandas Series (which is a single column), it prints in a more detailed but less "pretty" way. 

In the output immediately above, the code created a DataFrame with two columns (`trial` and `RT`) and then dropped the `trial` column, but as such it remained a DataFrame and so was nicely formatted.

In contrast, the output of using the `mean()` method on a single column (`RT`) in the cell above that is a Series.  
<p><p>
We'll worry about the formatting later, but it's good to understand why it happens because the distinction between DataFrames and Series often causes confusion and errors if it's not understood.
</div>